In [1]:
import os
import geocoder
import re

import dask
import dask.multiprocessing
import dask.dataframe as dd
from dask import delayed

import pandas as pd
import numpy as np

from src.utils import (
    extract_area_from_floorplan,
    find_postcode,
    extract_area_from_dataframe,
    extract_other_data_from_floorplan
)

In [2]:
house_pictures = os.listdir("media/house_pictures")
house_pictures = [file for file in house_pictures if file != ".DS_Store"]
for folder in house_pictures:
    files = os.listdir(f"media/house_pictures/{folder}")
    if not files:
        print(f"{folder} is empty")

In [3]:
# garden_mentions_garden = garden_data["key_features"].apply(
#     lambda x: any("garden" in item.lower() for item in x)
# )
# communal_garden_mentions_no_garden = garden_no_garden_data["key_features"].apply(
#     lambda x: any("communal garden" in item.lower() for item in x)
# )
# garden_mentions_no_garden = garden_no_garden_data["key_features"].apply(
#     lambda x: any("garden" in item.lower() for item in x)
# )

# print(f"Total garden mentions in garden data: {sum(garden_mentions_garden)}")
# print(f"Total communal garden mentions in no garden data: {sum(communal_garden_mentions_no_garden)}")
# print(f"Total garden mentions in no garden data: {sum(garden_mentions_no_garden)}")
# print(f"Shape of garden data is {garden_data.shape}")
# print(f"Shape of no garden data is {garden_no_garden_data.shape}")

In [4]:
files = os.listdir("data/August 2023/")
files = [file for file in files if file != ".DS_Store"]
postcodes_set = set()

for file in files:
    postcode = file.split("_")[-1].split(".")[0]
    postcodes_set.add(postcode)
postcode_list = list(postcodes_set)

# Create an empty DataFrame to store the output
all_data = pd.DataFrame()

# Iterate over each postcode in the postcode_list
for postcode in postcode_list:
    # Read the no garden data and garden data for the current postcode
    garden_no_garden_data = pd.read_parquet(f"data/August 2023/house_data_no_garden_{postcode}.parquet")
    garden_data = pd.read_parquet(f"data/August 2023/house_data_garden_{postcode}.parquet")

    # Merge the garden data and no garden data
    garden_data_id = garden_data["id"].tolist()
    no_garden_data = garden_no_garden_data[~garden_no_garden_data["id"].isin(garden_data_id)].copy()
    no_garden_data.reset_index(drop=True, inplace=True)
    no_garden_data["garden"] = 0
    garden_data["garden"] = 1

    merged_data = pd.concat([no_garden_data, garden_data])
    merged_data.drop_duplicates(subset=["id", "description", "address"], keep="last", inplace=True)
    
    # Append the current iteration of all_data to the output_data DataFrame
    all_data = pd.concat([merged_data, all_data])

# Reset the index of the output_data DataFrame
all_data.reset_index(drop=True, inplace=True)

In [5]:
# Turn numeric columns into numeric types
all_data["price"] = (
    all_data["price"].str.replace("£", "").str.replace(",", "").astype(int)
)
# Fill missing values with a default value (0 in this case)
all_data["bathrooms"] = all_data["bathrooms"].fillna("0")
all_data["bathrooms"] = all_data["bathrooms"].astype("int")
# Fill missing values with a default value (0 in this case)
all_data["bedrooms"] = all_data["bedrooms"].fillna("0")
all_data["bedrooms"] = all_data["bedrooms"].astype("int")
# Extract years left in tenure and fill missing values
all_data["tenure_lease_length"] = all_data["tenure_lease_length"].str.extract(r"(\d+)")
all_data["tenure_lease_length"] = all_data["tenure_lease_length"].fillna("0")
all_data["tenure_lease_length"] = all_data["tenure_lease_length"].astype("int")
# Extract the letter after "Band:" and handle special cases
all_data['extracted_band'] = all_data['council_tax_band'].str.extract(r'Band:\s*([A-Za-z]+)')
# Replace "Ask agent" in tenure type with None values
all_data["tenure"] = all_data["tenure"].replace("Ask agent", None)

In [6]:
# # First, extract the post code from the address    
# all_data["postcode"] = all_data["address"].apply(lambda x: x.split(",")[-1].replace(" ", ""))

# # If the postcode isn't contained in the address or it uses a different forma than Nx
# # or NWx, then use geocoder to extract the postcode
# mask_postcode = all_data["postcode"].str.len() > 4
# # Create a temporary dataframe to extract the postcode of the addresses in the mask
# no_postcode_data = all_data[mask_postcode].copy()
# no_postcode_data["road"] = no_postcode_data["address"].apply(lambda x: x.split(",")[0])
# no_postcode_data["postcode"] = no_postcode_data["road"].apply(find_postcode)
# # Copy the new postcode for the elements in the mask to the original dataframe
# all_data.loc[mask_postcode, "postcode"] = no_postcode_data["postcode"]

In [7]:
# # Assuming no_postcode_data is your DataFrame
# dask_dataframe = dd.from_pandas(no_postcode_data, npartitions=8)  # Adjust the number of partitions as needed

# # Apply the find_postcode function using Dask's map_partitions
# dask_dataframe["postcode"] = dask_dataframe.map_partitions(lambda df: df["road"].apply(find_postcode), meta=("postcode", "object"))

# # Compute the Dask DataFrame to get the final result
# result_df = dask_dataframe.compute()

In [8]:
# The data scraped from rightmove is in the format xxx sq. ft (xxx sq. m). We need to 
# extract the number before "sq. ft" using re
mask_size = ~all_data["size"].isna()
mask_no_size = all_data["size"].isna()
# Create a copy of the data with missing size information
size_data = all_data[mask_size].copy()
# Extract the size from the floorplan for the houses with missing size information
size_data["size"] = size_data["size"].apply(extract_area_from_dataframe)
all_data.loc[mask_size, "size"] = size_data["size"]
# In the size information wasn't available in rightmove, we need to extract the 
# text from the floorplan and then extract the size
# Create a copy of the data with missing size information
# no_size_data = all_data[mask_no_size].copy()
# no_size_data_test = all_data[mask_no_size].copy()
# Extract the size from the floorplan for the houses with missing size information
# no_size_data["size"] = no_size_data["id"].apply(extract_area_from_floorplan)
# Assuming no_postcode_data is your DataFrame
dask_dataframe = dd.from_pandas(all_data[mask_no_size], npartitions=8)  # Adjust the number of partitions as needed

# Apply the find_postcode function using Dask's map_partitions
dask_dataframe["size"] = dask_dataframe.map_partitions(lambda df: df["id"].apply(extract_area_from_floorplan), meta=("id", "object"))

# Compute the Dask DataFrame to get the final result
no_size_data = dask_dataframe.compute()
# Update the size information in the original dataframe
all_data.loc[mask_no_size, "size"] = no_size_data["size"]
all_data["size"] = all_data["size"].astype(float)

In [12]:
# Extract the type of outdoor space (if any) and the floor level from the floorplan
col_names = ["outdoor_type", "floor"]

# Create a Dask delayed computation for each image_id
delayed_results = [delayed(extract_other_data_from_floorplan)(image_id) for image_id in all_data["id"]]

# Compute the results using Dask multiprocessing
with dask.config.set(scheduler="processes"):
    results = dask.compute(*delayed_results)

# Update the DataFrame with computed results
outdoor_space_values = [result[0] for result in results]
floor_values = [result[1] for result in results]

all_data[col_names[0]] = outdoor_space_values
all_data[col_names[1]] = floor_values


In [59]:
print(sum(all_data["address"].str.contains("court", case=False)))
print(sum(all_data["address"].str.contains("estate", case=False)))
print(sum(all_data["address"].str.contains("house", case=False)))


55
12
32


In [60]:
all_data.loc[all_data["address"].str.contains("court", case=False)]

,id,price,added_reduced,address,description,price_change_date,price_change_value,type_house,bathrooms,bedrooms,...,close_stations,close_stations_type,tenure_ground_rent,tenure_annual_service_charge,tenure_lease_length,council_tax_band,garden,outdoor_type,floor,extracted_band
6,123836303,525000,Reduced on 21/06/2022,"Whittington Court, Aylmer Road, London, N2",An impressive large (1043 sq ft) 2 double bedr...,"[21/06/2022, 03/06/2022]","[Price changed from £550,000 to £525,000, Init...",Apartment,2,2,...,"[[East Finchley , 0.5 miles], [Highgate , 0.6 ...","[underground, underground, underground]",£50 per year,£3000,90,Band: E,0,None,None,E
17,128846540,500000,Added on 07/11/2022,"Aylmer Court, Sheldon Avenue, East Finchley, L...",Close to East Finchley and Highgate station is...,[],[],Apartment,2,3,...,"[[East Finchley , 0.5 miles], [Highgate , 0.5 ...","[underground, underground, underground]",£20 per year,£4058.6,123,Band: D,0,None,None,D
33,129170519,435000,Reduced on 03/07/2023,"Monarch Court, Hampstead Garden Suburb","A 2 double bedroom, second floor flat situated...",[],[],Apartment,1,2,...,"[[East Finchley , 0.4 miles], [Golders Green ,...","[underground, underground, underground]",Ask agent,Ask agent,0,Ask agent,0,balcony,second,0
42,133272632,399950,Reduced on 16/03/2023,"Risborough Court, Muswell Hill, London, N10",Situated within minutes walk of Muswell Hill B...,[],[],Apartment,1,2,...,"[[Highgate , 0.9 miles], [Alexandra Palace , 1...","[underground, nationalrail, underground]",£0 per year,£0,0,Band: TBC,0,None,None,0
49,138391784,365000,Added on 08/08/2023,"Chessing Court, Fortis Green, London, N2",This one bedroom penthouse apartment benefits ...,[],[],Apartment,1,1,...,"[[East Finchley , 0.3 miles], [Highgate , 1.0 ...","[underground, underground, underground]",£0 per year,£1440,116,Band: C,0,None,sixth,C
50,128563007,360000,Reduced on 01/06/2023,"Chessing Court, Fortis Green, London, N2",*BACK ON THE MARKET AS OF 21ST AUGUST 2023* Br...,[],[],Apartment,1,1,...,"[[East Finchley , 0.3 miles], [Highgate , 1.0 ...","[underground, underground, underground]",£0 per year,£1440,156,Band: C,0,None,second,C
67,134159906,475000,Reduced on 12/06/2023,"Lyttelton Court, Hampstead Garden Suburb, N2",A beautifully presented two double bedroom gro...,"[12/06/2023, 27/04/2023]","[Price changed from £495,000 to £475,000, Init...",Apartment,1,2,...,"[[East Finchley , 0.6 miles], [Golders Green ,...","[underground, underground, underground]",Ask agent,Ask agent,141,Ask agent,1,balcony,ground,0
73,138648161,450000,Added on 15/08/2023,"Widecombe Court, Lyttelton Road, East Finchley...",We are pleased to offer this well-presented tw...,[15/08/2023],"[Initial entry found: £450,000]",Apartment,1,2,...,"[[East Finchley , 0.4 miles], [Golders Green ,...","[underground, underground, underground]",£100 per year,£1630,91,Band: D,1,balcony,first,D
101,139032947,525000,Added on 24/08/2023,"Townshend Court, St Johns Wood, NW8","A well-proportioned, first floor one bedroom f...",[],[],Flat,1,1,...,"[[St. John's Wood , 0.4 miles], [Swiss Cottage...","[underground, underground, overground]",Ask agent,Ask agent,167,Band: D,0,None,first,D
125,138014363,500000,Reduced on 21/03/2023,"Frognal Court, Finchley Road NW3, NW3",A three double bedroom flat set on the second ...,[28/08/2023],"[Initial entry found: £500,000]",Block of Apartments,1,3,...,"[[Finchley Road & Frognal , 0.1 miles], [Finch...","[overground, underground, underground]",Ask agent,Ask agent,156,Band: D,0,None,ground,D


## Analysis

In [ ]:
# all_data["size_sqm"] = all_data["size"] * 0.092903
all_data["price_sqm"] = all_data["price"] / (all_data["size"] * 0.092903)

In [ ]:
# Count the occurrences of each postcode in the dataset
postcode_counts = all_data["postcode"].value_counts()
# Filter out postcodes that appear less than 5 times
postcode_counts_filtered = postcode_counts[postcode_counts >= 5]
# Get a list of the filtered postcodes
filtered_postcodes = postcode_counts_filtered.index.tolist()
# Filter the data based on the filtered postcodes
filtered_data = all_data[all_data["postcode"].isin(filtered_postcodes)]
# Define the columns to be used for analysis
cols_analysis = ["price", "price_sqm"]
# Group the filtered data by postcode and calculate the mean and median of the selected columns
analysis_results = filtered_data.groupby("postcode")[cols_analysis].aggregate(["mean", "median"]).round(1)
# Print the analysis results
analysis_results


In [ ]:
filtered_data[filtered_data["floor"].isna()][["id", "size", "floor"]].sample(15, random_state=42)

In [ ]:
price_outliers = filtered_data[(filtered_data["size"] < 275)]
price_outliers[["id","size", "floor"]]

In [ ]:
image_id = "138272951"
print(extract_other_data_from_floorplan(image_id))
command = f"open media/floorplans/{image_id}_floorplan.png"
! {command}

In [ ]:
all_data["garden"] == 1 & ~all_data["garden_floorplan"].isna()

In [ ]:
mask_garden = (all_data["garden"] == 1) & (all_data["garden_floorplan"].isna())
all_data[mask_garden][["id", "garden", "garden_floorplan"]]